Serveur chez OVH :
- Mongo DB
- Docker
- Appli de rajout d'élévation

Réunion du 27/10/2022

- Acces MongoDB via MongoDB Compass
- Interpolation : se donner une tolérance. Si le pointage au passage d'un arrêt est trop éloigné des coordonénes gps rajouter un enregistrement par interpolation
- Sauvegarder la donnée nettoyée sous forme brute dans la base MongoDB (collection)
- Dupliquer les lignes avec des destinations finales différentes


In [1]:
import pandas as pd
import glob

# Read files
Get the file names

In [2]:
filenames = glob.glob("./data/*")
filenames.sort()
len(filenames), filenames[0]

(6459, './data/20190115_1.csv')

There is 6459 files.

We make a single data frame using ``concatDf`` function :

- Records with ``lines==-1`` and ``lines>5``
- Records with ``state=='Non affecté'``,``state=='Délocalisé``
  and ``state='finalDestination'`` are removed
- We add the date in the format yyyymmdd using the file name.


In [3]:
def concatDf(filenames):
    list_for_concat = [] #instancier la liste
    #former la liste 'list_for_concat' avec tous les dataframes à concatener
    for f in filenames:     
        df_inter = pd.read_csv(f)
        df_inter = df_inter[(df_inter['ligne']!=-1)]     
        df_inter = df_inter[(df_inter['ligne']<=5)]
        df_inter = df_inter[(df_inter['state']!='Non affecté')]
        df_inter = df_inter[(df_inter['state']!='Délocalisé')]
        df_inter = df_inter[(df_inter['state']!='finalDestination')!='']
        df_inter['Year'] = f[7:11]
        list_for_concat.append(df_inter)
    df_final = pd.concat(list_for_concat)
    return df_final

def whitespace_remover(dataframe):
    # iterating over the columns
    for i in dataframe.columns:
        # checking datatype of each columns
        if dataframe[i].dtype == 'object':
            # applying strip function on column
            dataframe[i] = dataframe[i].map(str.strip)
    return dataframe

In [4]:
df_test = concatDf(filenames)
df_test = whitespace_remover(df_test)
len(df_test)

9232398

We get 9232398 records

In [5]:
df_test.head(62)

,ligne,date,hour,longitude,latitude,elevation,lastStop,finalDestination,state,Year
144,3,15/01/2019,08:31:54,47.6285,6.8596,355.0,Colbert,Gare TGV,En course,2019
145,3,15/01/2019,08:32:09,47.6288,6.8611,355.0,Colbert,Gare TGV,En course,2019
146,3,15/01/2019,08:32:39,47.6288,6.8614,356.0,Multiplexe,Gare TGV,En course,2019
147,3,15/01/2019,08:33:09,47.6286,6.8636,355.0,Multiplexe,Gare TGV,En course,2019
148,3,15/01/2019,08:33:39,47.6273,6.8643,355.7,Multiplexe,Gare TGV,En course,2019
...,...,...,...,...,...,...,...,...,...,...
201,3,15/01/2019,08:56:39,47.5846,6.8963,369.0,Gare TGV,Gare TGV,En course,2019
202,3,15/01/2019,08:57:09,47.5832,6.8971,369.8,Gare TGV,Gare TGV,En course,2019
203,3,15/01/2019,08:57:39,47.5804,6.8992,381.6,Gare TGV,Gare TGV,En course,2019
204,3,15/01/2019,08:58:08,47.5786,6.8982,380.0,1er RA,Gare TGV,En Terminus,2019


In [6]:
pd.unique(df_test['state'].sort_values(ascending=True))

array(['En Terminus', 'En course'], dtype=object)

# Filtering finalDestination

We work on the names of the final destination

- We correct manually some 'finalDestinations'
- We remove the records with no 'finalDestination'
- We remove the records with ``finalDestination=='Dépôt'``

In [7]:
pd.read_csv('./Optymo/line.csv')

,ligne,Directions
0,1,Valdoie Mairie;Résidences
1,2,Argésians;Justice;Bavilliers;Cravanche
2,3,Châtenois;Valdoie Mairie;Gare TGV;Liberté Fol...
3,4,Pépinière;Cerisiers;Froideval;Offemont
4,5,Éloie;Bois Joli;Prés d'Aumont
5,8,Liberté Follereau;La forêt
6,9,République


In [8]:
pd.unique(df_test['finalDestination'].sort_values(ascending=True))

array(['', 'Acacias', 'Argiésans', 'Bavilliers', 'Belfort Gare', 'Berger',
       'Bois Joli', 'Cerisiers', 'Chaume', 'Châtenois', 'Clémenceau',
       'Cravanche', 'Dépôt', 'Dépôt RTTB', 'Froideval par Pépinière',
       'Gare', 'Gare TGV', 'Gare de Belfort', 'Géhant',
       'Hauts de Belfort', 'IUT', 'Justice', 'LA DAME', 'La Douce',
       'Le Nôtre', 'Liberté Follereau', 'Liberté Madrid', 'Lycée Quelet',
       'Lycée Quelet par Valdoie', 'Morvillars Gare', 'Offemont',
       'Offemont Cerisier', 'Offemont Cerisiers', 'Offemont Moulin',
       "Prés D'Aumont", 'Pépinière', 'Résidences', 'Signoret',
       'Valdoie Mairie', 'Valdoie par Jaurès'], dtype=object)

In [9]:
 def Update_final_destination(df):
    df['finalDestination'].replace(to_replace='1re Armée',                value='Valdoie Mairie', inplace=True)
    df['finalDestination'].replace(to_replace='Acacias',                  value='Argésians', inplace=True)
    df['finalDestination'].replace(to_replace='Argiésans',                value='Argésians', inplace=True)
    df['finalDestination'].replace(to_replace='Berger',                   value='Froideval', inplace=True)
    df['finalDestination'].replace(to_replace='Chaume',                   value='Argésians', inplace=True)
    df['finalDestination'].replace(to_replace='Froideval par Pépinière',  value='Froideval', inplace=True)
    df['finalDestination'].replace(to_replace='Géhant',                   value='Châtenois', inplace=True)
    df['finalDestination'].replace(to_replace='Hauts de Belfort',         value='Justice', inplace=True)
    df['finalDestination'].replace(to_replace='LA DAME',                  value='Bavilliers', inplace=True)
    df['finalDestination'].replace(to_replace='La Douce',                 value='Résidences', inplace=True)
    df['finalDestination'].replace(to_replace='Le Nôtre',                 value='Résidences', inplace=True)
    df['finalDestination'].replace(to_replace='Liberté Madrid',           value='Liberté Follereau', inplace=True)
    df['finalDestination'].replace(to_replace='LIBERTE FOLLEREAU',        value='Liberté Follereau', inplace=True)
    df['finalDestination'].replace(to_replace='Lycée Quelet',             value='Valdoie Mairie', inplace=True)
    df['finalDestination'].replace(to_replace='Lycée Quelet par Valdoie', value='Valdoie Mairie', inplace=True)
    df['finalDestination'].replace(to_replace='Offemont Moulin',          value='Offemont', inplace=True)
    df['finalDestination'].replace(to_replace='Offemont Cerisier',        value='Cerisiers', inplace=True)
    df['finalDestination'].replace(to_replace='Signoret',                 value='Résidences', inplace=True)
    df['finalDestination'].replace(to_replace='Valdoie par Jaurès',       value='Valdoie Mairie', inplace=True)
    df['finalDestination'].replace(to_replace='Valdoie Blumberg',         value='Valdoie Mairie', inplace=True)
    
    df['finalDestination'].replace(to_replace='Dépôt RTTB', value='Dépôt', inplace=True)
    df['finalDestination'].replace(to_replace='DEPOT', value='Dépôt', inplace=True)
    
    
    return df

In [10]:
df_test = Update_final_destination(df_test)
df_test = df_test[(df_test['finalDestination']!='')]
df_test = df_test[(df_test['finalDestination']!='Dépôt')]
pd.unique(df_test['finalDestination'].sort_values(ascending=True))

array(['Argésians', 'Bavilliers', 'Belfort Gare', 'Bois Joli',
       'Cerisiers', 'Châtenois', 'Clémenceau', 'Cravanche', 'Froideval',
       'Gare', 'Gare TGV', 'Gare de Belfort', 'IUT', 'Justice',
       'Liberté Follereau', 'Morvillars Gare', 'Offemont',
       'Offemont Cerisiers', "Prés D'Aumont", 'Pépinière', 'Résidences',
       'Valdoie Mairie'], dtype=object)

There is some destinations to check manually
- 'Morvillars Gare' does not exist anymore (see report)
- 'Offemont Cerisiers' is either 'Offemont' or 'Cerisiers'

# Read files with stop names

In [11]:
file_arrets = "./Optymo/stops&GPS.xlsx"
df_stops = pd.read_excel(file_arrets, sheet_name=0, header=[0])
stop_names = pd.unique(df_stops['stop_name'].sort_values(ascending=True))
stop_names

array(['1er RA', '1re Armée', '1ère Dfl', '4 As', '5 Fontaines',
       '5ème Db', 'Acacias', 'Ackermann', 'Acropole', 'Adour', 'Allaine',
       'Alsace', 'Alstom Etang', 'Altkirch', 'Ancienne Douane',
       'Ancienne Gare', 'Ancienne École', 'Andelnans', 'Andelnans Prés',
       'Angeot', 'Angoulotte', 'Anjoutey', 'Ap90', 'Arcades', 'Argiésans',
       'As de Trèfle', 'Atria', 'Aubépines', 'Autrechêne', 'Auxelles-Bas',
       'Auxelles-Haut', 'Aéroparc', 'Aéroparc 2', 'Aéroparc 3', 'Badevel',
       'Ballinamuck', 'Ballon', 'Bambois', 'Banvillars', 'Barbier',
       'Barrière Blanche', 'Bas du Village', 'Bascule', 'Batterie',
       'Bavilliers', 'Bavière', 'Beaucourt', 'Bellevue', 'Benoît Frachon',
       'Berger', 'Berly', 'Bermont', 'Bessoncourt', 'Beucler',
       'Beurrerie', 'Bichat', 'Blessonniers', 'Blum', 'Bochets', 'Bohn',
       'Bois Joli', 'Bois Sarclés', "Bois d'Essert", 'Bon Bois',
       'Bonnaymé', 'Bonneff', 'Boron', 'Bosmont', 'Botans', 'Boulloche',
       'Bouloy

In [12]:
df_stops.head()

,stop_name,stop_lat,Unnamed: 2,stop_name.1,stop_lon
0,1er RA,47.578685,NaN,1er RA,6.898170
1,1re Armée,47.655007,NaN,1re Armée,6.847505
2,1ère Dfl,47.748439,NaN,1ère Dfl,6.820597
3,4 As,47.638071,NaN,4 As,6.855129
4,5 Fontaines,47.507894,NaN,5 Fontaines,7.005233


# Filtering lastStop

We filter the names of the lastStop which are not referenced in the file ``stops&GPS.xlsx``

#### Compare last stop with stop_names
We get names with difference with the _official_ name

In [13]:
lastStop = pd.unique(df_test['lastStop'].sort_values(ascending=True))
lastStop

array(['', '1er RA', '1re Armée', '4 As', 'Acacias', 'Alstom Etang',
       'Altkirch', 'Andelnans Prés', 'Argiésans centre', 'Aubépines',
       'Ballinamuck', 'Ballon', 'Bascule', 'Bavilliers', 'Benoit Frachon',
       'Berger', 'Beurrerie', 'Bichat', 'Blum', 'Bohn', 'Bois Joli',
       'Bonneff', 'Bosmont', 'Bourogne', 'Briand', 'Bruxelles', 'CFA',
       'Camus', 'Canal', 'Carrières', 'Centre Commercial', 'Cerisiers',
       'Champ de Mars', 'Chemin de la ferme', 'Châtenois',
       'Châtenois Forges', 'Chênois', 'Cimetière militaire', 'Clémenceau',
       'Colbert', 'Colmar', 'Conforama', 'Courbet', 'De Gaulle',
       'Denfert Rochereau', 'Dubail', 'Dépôt RTTB', 'Eglise', 'Eloie',
       'Eloie Chaume', 'Eluard', 'Espace Gantner', 'Essert', 'Ferrette',
       'Fg de Lyon', 'Foch', 'Gardey', 'Gare', 'Gare TGV', "Grand'Combe",
       'Grottes', 'Guidon', 'Géhant', 'Hatry', 'Hauts de Belfort',
       'Hôpital', 'Hôpital NFC', 'IUT', 'Jacquot', 'James Long', 'Jaurès',
       'Jaurès 

In [14]:
import numpy as np
np.setdiff1d(lastStop, stop_names, assume_unique=False)

array(['', 'Argiésans centre', 'Benoit Frachon', 'Chemin de la ferme',
       'Cimetière militaire', 'Dépôt RTTB', 'Eglise', 'Eloie Chaume',
       'Hôpital', 'Hôpital NFC', 'Jaurès', 'Jaurès Hôpital', "L'assise",
       'LA DAME', "Prés D'Aumont", 'Rte de Bermont', "Techn'hom 1/UTBM",
       "Techn'hom 2", "Techn'hom 3", "Techn'hom Cravanche"], dtype=object)

####  Filtering lastStop manually

- We filter manually the unknown names
- We remove the records with empty ``lastStop``
- We remove the record with ``lastStop=='Dépôt'``
- We remove the record with ``lastStop=='Eglise'``

``'Eglise'`` does not exist anymore (see report)


In [15]:
def Update_last_stop(df):
    df['lastStop'].replace(to_replace='Argiésans centre',   value='Argiésans', inplace=True)
    df['lastStop'].replace(to_replace='Bas du village',     value='Bas du Village', inplace=True)
    df['lastStop'].replace(to_replace='Benoit Frachon',     value='Benoît Frachon', inplace=True)
    df['lastStop'].replace(to_replace='Chemin de la ferme', value='Chemin de la Ferme', inplace=True)
    df['lastStop'].replace(to_replace='Cimetière militaire',value='Cimetière Militaire', inplace=True)
    df['lastStop'].replace(to_replace='Cravanche centre',   value='Cravanche Centre', inplace=True)
    df['lastStop'].replace(to_replace='Dépôt RTTB',         value='Dépôt', inplace=True)
#    df['lastStop'].replace(to_replace='Eglise',         value='???'', inplace=True) # Église de Grosne ou Fèche l'église ?
    df['lastStop'].replace(to_replace='Eloie Chaume',       value='Eloie', inplace=True)
#    df['lastStop'].replace(to_replace='Haut des Prés',        value='???', inplace=True)
    df['lastStop'].replace(to_replace='Hôpital',            value='Hôpital Nfc', inplace=True)
    df['lastStop'].replace(to_replace='Hôpital NFC',        value='Hôpital Nfc', inplace=True)
    df['lastStop'].replace(to_replace='Jaurès',             value='Jean Jaurès', inplace=True)
    df['lastStop'].replace(to_replace='Jaurès Hôpital',     value='Jean Jaurès', inplace=True)
    df['lastStop'].replace(to_replace="L'assise",           value="Froideval l'Assise", inplace=True)
    df['lastStop'].replace(to_replace='LA DAME',            value='La Dame', inplace=True)
#    df['lastStop'].replace(to_replace='Les Etangs',           value='????', inplace=True)
#    df['lastStop'].replace(to_replace='Nautibus',             value='???', inplace=True)
    df['lastStop'].replace(to_replace="Prés D'Aumont",      value="Prés d'Aumont", inplace=True)
    df['lastStop'].replace(to_replace="Rte de Bermont",     value="Route de Bermont", inplace=True)
#    df['lastStop'].replace(to_replace="Schweitzer",      value="???", inplace=True)
    df['lastStop'].replace(to_replace="Techn'hom 1/UTBM",   value="Techn'Hom 1/UTBM", inplace=True)
    df['lastStop'].replace(to_replace="Techn'hom 2",        value="Techn'Hom 2", inplace=True)
    df['lastStop'].replace(to_replace="Techn'hom 3",        value="Techn'Hom 3", inplace=True)
    df['lastStop'].replace(to_replace="Techn'hom Cravanche",value="Techn'Hom Cravanche", inplace=True)
    
    return df

In [16]:
df_test = Update_last_stop(df_test)
df_test = df_test[(df_test['lastStop']!='')]
df_test = df_test[(df_test['lastStop']!='Dépôt')]
df_test = df_test[(df_test['lastStop']!='Eglise')]
lastStop_filtered = pd.unique(df_test['lastStop'].sort_values(ascending=True))

In [17]:
np.setdiff1d(lastStop_filtered, stop_names, assume_unique=False)

array([], dtype=object)

# Save Courses


In [18]:
len(df_test)

9018193

In [19]:
#df_test.to_csv("filtered_data-2022-11-21.csv")

# Try by year
grouped = df_test.groupby("Year")
grouped

In [20]:
for name, group in grouped:
    group.to_csv("filtered_data_{}".format(name))